In [1]:
import pyspark_cassandra

In [2]:
cf_transaksi_standard = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="cf_transaksi_standard", keyspace="skripsi").load()
cf_buku = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="cf_buku", keyspace="skripsi").load()
cf_transaksi_jurusan_kategori = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="cf_transaksi_jurusan_kategori", keyspace="skripsi").load()
mv_transaksi_kondisi_koleksi = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="mv_transaksi_kondisi_koleksi", keyspace="skripsi").load()
cf_usulan = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="cf_usulan", keyspace="skripsi").load()

In [3]:
cf_transaksi_standard.registerTempTable("cf_transaksi_standard")
sqlContext.sql("select * from cf_transaksi_standard where MONTH(tanggal_pinjam)=6 and DAY(tanggal_pinjam)=30 and semester=1 LIMIT 5").show()

+--------------+-------------+---------------+---------+-----+--------+------------+------------+------------+--------+------------+--------+
|tanggal_pinjam|tanggal_batas|tanggal_kembali|unique_id|denda|fakultas|id_transaksi|kode_anggota|nama_jurusan|semester|tahun_ajaran|terbayar|
+--------------+-------------+---------------+---------+-----+--------+------------+------------+------------+--------+------------+--------+
+--------------+-------------+---------------+---------+-----+--------+------------+------------+------------+--------+------------+--------+



In [4]:
#query 1
cf_transaksi_standard.registerTempTable("cf_transaksi_standard")
sqlContext.sql("""WITH cte as (
                    SELECT kode_anggota,
                           nama_jurusan,
                           fakultas,
                           tahun_ajaran, 
                           COUNT(unique_id) AS jumlah,
                           ROW_NUMBER() OVER (PARTITION BY tahun_ajaran ORDER BY COUNT(unique_id) DESC) AS r 
                    FROM cf_transaksi_standard
                    WHERE tahun_ajaran IN ('2014/2015','2015/2016','2016/2017')
                    GROUP BY kode_anggota,nama_jurusan,fakultas,tahun_ajaran
                )
                SELECT kode_anggota,nama_jurusan,fakultas,tahun_ajaran,jumlah FROM cte where r<=3 
                ORDER BY tahun_ajaran,jumlah""").show()

+------------+--------------+--------+------------+------+
|kode_anggota|  nama_jurusan|fakultas|tahun_ajaran|jumlah|
+------------+--------------+--------+------------+------+
|     MP15030|      karyawan|karyawan|   2014/2015|   353|
|      94-029|      karyawan|karyawan|   2014/2015|   325|
|    11412046|Sastra Inggris|  Sastra|   2014/2015|   339|
|      00-007|      karyawan|karyawan|   2015/2016|   460|
|       46121|      karyawan|karyawan|   2015/2016|   297|
|     MP15030|      karyawan|karyawan|   2015/2016|   390|
|      98-011|      karyawan|karyawan|   2016/2017|   162|
|      86-002|      karyawan|karyawan|   2016/2017|   142|
|      03-030|      karyawan|karyawan|   2016/2017|   128|
+------------+--------------+--------+------------+------+



In [5]:
#query 2
cf_transaksi_standard.registerTempTable("cf_transaksi_standard")
sqlContext.sql("""SELECT YEAR(tanggal_pinjam) AS tahun,
                         MONTH(tanggal_pinjam) AS Bulan, 
                         SUM(denda) AS Jumlah, 
                         SUM(denda-terbayar) AS Belum_Terbayar
                    FROM cf_transaksi_standard
                    WHERE tahun_ajaran='2015/2016'
                    GROUP BY tahun,bulan
                    ORDER BY tahun,bulan""").show()

+-----+-----+---------+--------------+
|tahun|Bulan|   Jumlah|Belum_Terbayar|
+-----+-----+---------+--------------+
| 2015|    7|3826000.0|      944000.0|
| 2015|    8| 3.8554E7|      3.0562E7|
| 2015|    9|   7.39E7|      6.1746E7|
| 2015|   10|9000000.0|      810000.0|
| 2015|   11|6482000.0|      482000.0|
| 2015|   12|3086000.0|      326000.0|
| 2016|    1|5700000.0|     2816000.0|
| 2016|    2|8748000.0|     1568000.0|
| 2016|    3| 1.2256E7|     1992000.0|
| 2016|    4| 1.0536E7|     1742000.0|
| 2016|    5|7338000.0|      886000.0|
| 2016|    6|4766000.0|      214000.0|
+-----+-----+---------+--------------+



In [6]:
#query 3
cf_buku.registerTempTable('cf_buku')
sqlContext.sql("""WITH cte AS (
                    SELECT Jumlah_Sekarang,bulan,tahun,
                    SUM(Jumlah_Sekarang) OVER (ORDER BY tahun,bulan) AS Total_Sekarang 
                    FROM (
                        SELECT COUNT(unique_id) AS Jumlah_Sekarang,
                        MONTH(tanggal_datang) AS bulan, YEAR(tanggal_datang) AS tahun
                        FROM cf_buku
                        WHERE YEAR(tanggal_datang)!=1
                        GROUP BY bulan,tahun
                    ) AS total_per_bulan_tahun
                    GROUP BY bulan,tahun,Jumlah_Sekarang
                )
                SELECT bulan, tahun, Total_Sekarang,
                Total_Sekarang - LAG(Total_Sekarang,1) OVER (PARTITION BY bulan ORDER BY tahun) AS Kenaikan
                FROM cte WHERE tahun BETWEEN 2014 AND 2016
                GROUP BY Total_Sekarang,bulan,tahun
                ORDER BY bulan,tahun""").show()

+-----+-----+--------------+--------+
|bulan|tahun|Total_Sekarang|Kenaikan|
+-----+-----+--------------+--------+
|    1| 2014|        155703|    null|
|    1| 2015|        159300|    3597|
|    1| 2016|        162648|    3348|
|    2| 2014|        155888|    null|
|    2| 2015|        159497|    3609|
|    2| 2016|        162867|    3370|
|    3| 2014|        156355|    null|
|    3| 2015|        159852|    3497|
|    3| 2016|        163040|    3188|
|    4| 2014|        156803|    null|
|    4| 2015|        160222|    3419|
|    4| 2016|        163629|    3407|
|    5| 2014|        156979|    null|
|    5| 2015|        160505|    3526|
|    5| 2016|        163860|    3355|
|    6| 2014|        157090|    null|
|    6| 2015|        160695|    3605|
|    6| 2016|        164164|    3469|
|    7| 2014|        157337|    null|
|    7| 2015|        160917|    3580|
+-----+-----+--------------+--------+
only showing top 20 rows



In [7]:
#query 4
cf_transaksi_jurusan_kategori.registerTempTable('cf_transaksi_jurusan_kategori')
sqlContext.sql("""SELECT judul,
                         nama_koleksi,
                         status_sekarang,
                         COUNT(unique_id) AS jumlah
                    FROM cf_transaksi_jurusan_kategori
                    WHERE nama_jurusan='Teknik Arsitektur'
                    AND tahun_ajaran BETWEEN '2013/2014' AND '2015/2016'
                    AND kelompok_kategori IN ('Geologi','Tata Kota dan Pertamanan')
                    GROUP BY judul,status_sekarang,nama_koleksi
                    ORDER BY COUNT(unique_id) DESC 
                    LIMIT 5""").show()

+--------------------+------------+---------------+------+
|               judul|nama_koleksi|status_sekarang|jumlah|
+--------------------+------------+---------------+------+
|Brown skin, white...|        none|           baik|    44|
|Actionable web an...|        none|           baik|    39|
|Untaian ekonomi m...|        none|           baik|    34|
|Pelajaran cinta d...|        none|           baik|    33|
|Tata ruang air tanah|        none|           baik|    32|
+--------------------+------------+---------------+------+



In [8]:
#query 5 
cf_transaksi_jurusan_kategori.registerTempTable('cf_transaksi_jurusan_kategori')
sqlContext.sql("""SELECT tahun_ajaran,
                         semester,
                         COUNT(unique_id) AS Jumlah,
                         COUNT(unique_id) - FLOOR(AVG(COUNT(unique_id)) 
                         OVER (PARTITION BY tahun_ajaran ORDER BY tahun_ajaran)) 
                    AS Selisih_dengan_Rata_rata_tahunan
                    FROM cf_transaksi_jurusan_kategori
                    WHERE nama_jurusan='Teknik Industri' AND tahun_ajaran IN ('2013/2014','2014/2015','2015/2016')
                    GROUP BY tahun_ajaran,semester
                    ORDER BY tahun_ajaran,semester""").show()

+------------+--------+------+--------------------------------+
|tahun_ajaran|semester|Jumlah|Selisih_dengan_Rata_rata_tahunan|
+------------+--------+------+--------------------------------+
|   2013/2014|       1|  1612|                              31|
|   2013/2014|       2|  1551|                             -30|
|   2014/2015|       1|   855|                             -68|
|   2014/2015|       2|   992|                              69|
|   2015/2016|       1|   702|                             -97|
|   2015/2016|       2|   897|                              98|
+------------+--------+------+--------------------------------+



In [9]:
#query 6 => failed
mv_transaksi_kondisi_koleksi.registerTempTable('mv_transaksi_kondisi_koleksi')
sqlContext.sql("""SELECT kelompok_kategori, 
                         judul, 
                         kode_judul, 
                         kode_buku, 
                         COUNT(unique_id) AS Jumlah
                    FROM mv_transaksi_kondisi_koleksi
                    WHERE nama_koleksi IN ('Referensi','Laporan Kerja Praktek') 
                    AND tahun_ajaran IN ('2014/2015','2015/2016')
                    AND status_sekarang='hilang'
                    GROUP BY kelompok_kategori,judul,kode_judul,kode_buku
                    ORDER BY Jumlah DESC """).show()

+--------------------+--------------------+----------+---------+------+
|   kelompok_kategori|               judul|kode_judul|kode_buku|Jumlah|
+--------------------+--------------------+----------+---------+------+
|            Bangunan|Research methodology|    130182| 01872/13|     4|
|           Manajemen|Operating small s...|   2209236| 02925/07|     2|
|      Bahasa Inggris|Kamus Inggris Ind...|     15467| 00632/93|     2|
|            Bangunan|Morphological and...|   2186491| 02428/05|     2|
|     Teknologi Kimia|Pengetahuan keilm...|   2213244| 01940/14|     2|
|Pembuatan Produk-...|Management inform...|     43834| 04146/94|     2|
+--------------------+--------------------+----------+---------+------+



In [10]:
#query 7
#date_from=datetime.to_date(lit('2013-01-01')).cast(TimestampType())
"""cf_usulan.select('penerbit','count(unique_id)').\
                    filter('cf_usulan.tgl_usulan>="2013-01-01"').\
                    filter(cf_usulan.penerbit!='unknown').\
                    filter(cf_usulan.status=='Buku/AV Sedang Diolah').\
                    groupBy(lambda row: row.penerbit).\
                    reduceByKey(add).\
                    sort(desc('count(unique_id)')).\
                    limit(10).\
                    show()"""

cf_usulan.registerTempTable("cf_usulan")
sqlContext.sql("""SELECT penerbit,
                         COUNT(unique_id) AS Jumlah
                    FROM cf_usulan 
                    WHERE tgl_usulan>'2013-01-01' AND 
                    status='Buku/AV Sedang Diolah' AND 
                    penerbit!='unknown' 
                    GROUP BY penerbit 
                    ORDER BY Jumlah DESC
                    LIMIT 10""").show()#.explain()

+--------------------+------+
|            penerbit|Jumlah|
+--------------------+------+
| Woodhead Publishing|    22|
|Beijing Language ...|    12|
|        ALA Editions|    10|
|               Wiley|     8|
|                  FP|     6|
|             Pearson|     5|
|                Sage|     5|
|    ALA Neal-Schuman|     5|
|     Fairchild Books|     5|
|       PPM Manajemen|     4|
+--------------------+------+

